In [3]:
# import pandas
import pandas as pd

# import plotly
import plotly.express as px
from plotly.subplots import make_subplots

In [12]:
#%%  Reading PKL file into a DataFrame

# Reading the pickle files into DataFrames
ethereum_tx = pd.read_pickle('files\\ethereum_tx.pkl')
bitcoin_tx = pd.read_pickle('files\\bitcoin_tx.pkl')
ripple_tx = pd.read_pickle('files\\ripple_tx.pkl')
bnb_smart_tx = pd.read_pickle('files\\bnb_smart_tx.pkl')
avalanche_tx = pd.read_pickle('files\\avalanche_tx.pkl')

# Read the Binance transaction data and assign it to bnb_beacon_tx
binance_tx = pd.read_pickle('files\\binance_tx.pkl')
bnb_beacon_tx = binance_tx
bnb_beacon_tx['chain'] = 'bnb_beacon'

In [28]:
# assume both have the same columns: ['chain','date','tx_count']
web3_tx = pd.concat(
    [ethereum_tx, bitcoin_tx, ripple_tx, bnb_smart_tx, bnb_beacon_tx, avalanche_tx],
    ignore_index=True
)

# Convert date to datetime and chain to categorical
web3_tx['chain'] = pd.Categorical(web3_tx['chain'])
web3_tx = web3_tx.sort_values(['chain','date']).reset_index(drop=True)

# group by chain + week, summing tx_count
web3_tx = (web3_tx.groupby(['chain',pd.Grouper(key='date', freq='ME')])['tx_count'].sum().reset_index(name='tx_count'))
latest_month = web3_tx['date'].max()

# filter out rows belonging to the last week
web3_tx = web3_tx[web3_tx['date'] < latest_month].reset_index(drop=True)

# Exclude rows with a tx_count of 0
web3_tx = web3_tx[web3_tx['tx_count'] != 0]

# Calculate the total transactions per date and add it as a new column
web3_tx['total_tx'] = web3_tx.groupby('date')['tx_count'].transform('sum')

# Save and load the DataFrame
web3_tx = pd.DataFrame.to_pickle(web3_tx, 'files\\binance_tx.pkl')
web3_tx = pd.read_pickle('files\\web3_tx.pkl')
print(web3_tx.info())

<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 32 to 527
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   chain     439 non-null    category      
 1   date      439 non-null    datetime64[ns]
 2   tx_count  439 non-null    int64         
 3   total_tx  439 non-null    int64         
dtypes: category(1), datetime64[ns](1), int64(2)
memory usage: 14.2 KB
None


C:\Users\Matheus\AppData\Local\Temp\ipykernel_32296\1359907894.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# Create the figure
fig1 = px.bar(
    web3_tx,
    x='date',
    y='tx_count',
    color='chain',
    custom_data=['chain', 'tx_count', 'total_tx'],
    title='Blockchain Monthly Activity',
    labels={
        'date': 'Date',
        'tx_count': 'Transactions',
        'chain': 'Blockchain',
    },
    color_discrete_map={
        'bitcoin': '#ff9b00',  # Bitcoin Orange
        'ethereum': '#4750ff',  # Ethereum Blue
        'bnb_smart': '#ffe322',  # Binance Yellow
        'bnb_beacon': '#786834',  # Binance Yellow
        'ripple': '#DCDCDC',  # Ripple Grey
        'avalanche': '#ff1d00',  # Avalanche Red
    },
   
)

# Update traces showing chain, individual transactions, and total transactions
fig1.update_traces(
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Transactions: %{customdata[1]:,.0f}<br>" +
                  "Total (TX): %{customdata[2]:,.0f}<br>" +
                  "<extra></extra>"
)

fig1.update_layout(
    template="plotly_dark",              # inherit base dark theme 
    hovermode="x unified",
    dragmode="pan",
    title=dict(font=dict(size=14, color='white'), xanchor="left"),
    font=dict(family="IBM Plex Sans", color="white"),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.1), type="date"),
    margin=dict(l=10, r=10, t=40, b=10),
    yaxis=dict(title="Transactions"),
    height=500, width=705,
    barmode='stack',
    bargap=0.1,
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# get the full HTML as a string, with Plotly.js from CDN
fig1html = fig1.to_html(include_plotlyjs="cdn", full_html=True)

# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig1html = fig1html.replace("<head>", f"<head>\n    {font_link}")

# save the HTML to a file
with open("plots/html/blockchain_activity.html", "w") as f:
    f.write(fig1html)

<>:2: SyntaxWarning:

invalid escape sequence '\w'

<>:2: SyntaxWarning:

invalid escape sequence '\w'

C:\Users\Matheus\AppData\Local\Temp\ipykernel_32296\2049700646.py:2: SyntaxWarning:

invalid escape sequence '\w'

